<a href="https://colab.research.google.com/github/Asato4931/Asato4931-CA_5_1-STS-Task-JSONL/blob/main/5_1_STS_Task_%2B_JSON_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer

from sentence_transformers import SentenceTransformer
from sentence_transformers import models
 
import torch
import numpy as np

import pandas as pd


from scipy import stats

# Load pre-trained tokenizer
transformer = models.Transformer("cl-tohoku/bert-base-japanese-v2")
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")
pooling = models.Pooling(transformer.get_word_embedding_dimension(), pooling_mode_mean_tokens=True, pooling_mode_cls_token=False, pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[transformer, pooling])


valid_data = pd.read_json('/content/drive/MyDrive/Colab Notebooks/CA課題用/CA課題5/valid-v1.1.json', lines=True)


valid_data = valid_data.loc[:,['sentence1','sentence2']]


pearson = []
spearman = []




#Pearson

for i in range(0,1457):
  vector1_p = model.encode(valid_data.loc[i,'sentence1'])
  vector2_p = model.encode(valid_data.loc[i,'sentence2'])

  result_p= stats.pearsonr(vector1_p,vector2_p)
  pearson.append(result_p.statistic)


#Spearman

for i in range(0,1457):
  vector1_s = model.encode(valid_data.loc[i,'sentence1'])
  vector2_s = model.encode(valid_data.loc[i,'sentence2'])

  vector1_s = np.array(vector1_s).reshape(1,-1)
  vector2_s = np.array(vector2_s).reshape(1,-1)
  result_s = stats.spearmanr(vector1_s,vector2_s ,axis=None)
  spearman.append(result_s.statistic)

pearson_df = pd.DataFrame(pearson)
pearson_df = pearson_df * 5
pearson_df["metrics"] = "pearson"
pearson_df = pearson_df.rename(columns= { pearson_df.columns[0]: "score"})
pearson_df = pearson_df.reindex(columns=['metrics', 'score'])



spearman_df = pd.DataFrame(spearman)
spearman_df = spearman_df * 5
spearman_df["metrics"] = "spearman"
spearman_df = spearman_df.rename(columns= { spearman_df.columns[0]: "score"})
spearman_df = spearman_df.reindex(columns=['metrics', 'score'])


frames = [pearson_df,spearman_df]

result_5_1 = pd.concat(frames)

result_5_1.to_json('/content/drive/MyDrive/Colab Notebooks/CA課題用/CA課題5/CA_5_1 STS Task Results.jsonl', force_ascii=False, lines = True, orient='records' )


In [ ]:
!pip install -U sentence-transformers

!apt-get install mecab mecab-ipadic-utf8 python-mecab libmecab-dev
!pip install mecab-python3 

!pip install fugashi

!pip install unidic_lite